In [ ]:
import numpy as np
import os.path
import pandas
import matplotlib
import scipy.signal
import scipy.constants

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import gridspec

# import mpld3
# from mpld3 import plugins


# New version of this notebook
# First todo: clean up code. This mess is likely to introduce a mistake.



## Functions to load xviewer csv (from binary scope data) and scope csv

In [ ]:
def read_scope_xviewer_csv(filename):
    """ Read csv files exported using Yokogawa XViewer software
        
        Update 11-09-2017: auto-detect number of columns.        
    """
    headerRows = 10        
    with open(filename) as f:
        for x in range(headerRows):
            line = f.readline()            
            if "HResolution" in line:
                i1 = line.find(',')+1
                i2 = line.find(',',i1)                
                timeResolution = float( line[i1:i2].strip(' \t\n\r') )        
                numberOfCols = line.count(',')    
    data = pandas.read_csv(filename,header=None,skiprows=headerRows,usecols=range(1,numberOfCols)).values
    return data, timeResolution


def read_scope_csv(filename):
    """ Read csv files as stored by Yokogawa Ossiloscope.                     
    """    
    headerRows = 16        
    with open(filename) as f:
        for x in range(headerRows):
            line = f.readline()            
            if "HResolution" in line:                
                timeResolution = float( line[line.rfind(',')+1:-1].strip(' \t\n\r') )    
    data = pandas.read_csv(filename,header=None,skiprows=headerRows).values[:,1:-1]
    return data, timeResolution



## Functions to analize vibration time-traces

In [ ]:
from scipy.optimize import minimize
from scipy import interpolate

def lorentzian(x,p):    
    """
    Lorenzian function p[0] is center, p[1] is fwhm, p[2] is maximum intensity
    Similar to http://mathworld.wolfram.com/LorentzianFunction.html, but with p[2] maximum intensity, NOT integrated intensity of one.
    """
    return p[2] * (0.5 * p[1])**2 / ( ( x - p[0] )**2 + ( 0.5 * p[1] )**2 )

def lorentz_find_x(lorentzFwhm, lorentzMax, lorentzValue):    
    """ For lorenzian function y=f(x) with lorentzFwhm and lorentzMax centered at x=0, find x for y=lorentzValue. """
    def fun(x):
        return ( lorentzian( x, [0, lorentzFwhm, lorentzMax] ) -  lorentzValue ) ** 2    
    res = minimize( fun, lorentzFwhm,  method='Nelder-Mead', bounds=( (0, None ), ) )
    if not( res.success ):
        print 'lorentzFindX convergence error: ', res.message       
    return res.x[0]
    
def lorentz_find_flank_steepness(lorentzFwhm, lorentzMax, x):    
    """ Return steepness of flank for of lorentz dy/dx for given x. Assume lorentz centered at x=0. """  
    return lorentzMax * 8.0 * lorentzFwhm**2 * x / ( 4.0 * x**2 + lorentzFwhm**2 )**2
    

def linear_voltage_to_detuning(d, t, lorentzFwhm, lorentzMaxV):
    flankSteepness = lorentz_find_flank_steepness( lorentzFwhm, lorentzMaxV, lorentz_find_x( lorentzFwhm, lorentzMaxV, meanVInD ) )
    dHz = d / flankSteepness    
    return dHz

def rms_deviation( x ):
    """ return root-mean-square value of signal, with mean substracted """
    return np.sqrt(np.mean( (x - np.mean(x))**2))
    
def lorentz_voltage_to_detuning(d, t, lorentzFwhm, lorentzMaxV, doPlot=False ):    
    meanVInD = np.mean( d ) 
    minVInD = np.amin(d)    
    # Compute vibrations in Hz by first applying lorentzian function as lookup table V -> dHz
    minDFreqInD = lorentz_find_x( lorentzFwhm, lorentzMaxV, minVInD )        
    lorentzDFreq = np.linspace( -minDFreqInD-1e9, 0, 10000 )
    lorentzV = lorentzian( lorentzDFreq, [0, lorentzFwhm, lorentzMaxV] )
    dHz = interpolate.interp1d(lorentzV, lorentzDFreq)(d)  # will cause error if signal > lorentzMaxV  or if something went wrong with minDFreqInD
    # Print and plot results
    if doPlot:   
        i1 = 0
        i2 = min( 50000, len(d) )
        fig = plt.figure(figsize=(14, 6))
        plt.subplot(131)
        plt.plot( t[i1:i2], d[i1:i2] )
        plt.xlabel( 'time (s)' )
        plt.ylabel( 'transmission (V)' )
        plt.grid()
        plt.subplot(132)
        plt.plot( t[i1:i2], dHz[i1:i2]*1e-9 )
        plt.xlabel( 'time (s)' )
        plt.ylabel( 'cavity detuning (GHz)' )
        plt.grid()
        plt.subplot(133)
        plt.plot( lorentzDFreq*1e-9, lorentzV )
        plt.xlabel( 'cavity detuning GHz' )
        plt.ylabel( 'V' )
        plt.grid()
        plt.show()
    return dHz
            

## Measurements 14 September 2017 - Cavity length 20 um

In [ ]:
# Load files in memory.

dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170914/scope'
fileNames = [ 'SCOPE_013.wdf.csv', 'SCOPE_015.wdf.csv', 'SCOPE_017.wdf.csv', 'SCOPE_019.wdf.csv' ]
dataDict = {}

for fileName in fileNames:
    if fileName in dataDict:
        continue
    data, timeResolution = read_scope_xviewer_csv( os.path.join(dataDir,fileName) )
    time = timeResolution * np.arange(data.shape[0]) 
    print 'time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
        timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )   
    dataDict[fileName] = data


In [ ]:
# Plot first check of files

for fileName in fileNames:    
    data = dataDict[fileName]
    i1 = 0
    i2 = np.argmax( time > 1 )
    plt.figure(figsize=(16, 8))
    plt.plot( time[i1:i2], data[i1:i2,0] )
    plt.grid()
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'V' )
    plt.title( fileName )
    plt.show()  

for fileName in fileNames:    
    data = dataDict[fileName]    
    plt.hist(data[:,0],20,label=fileName)
plt.legend()
plt.grid()
plt.show()

for fileName in fileNames:    
    data = dataDict[fileName]    
    print 'filename {}, min {} V, max {} V'.format( fileName, np.amin( data[:,0] ), np.amax( data[:,0] ) )    


In [ ]:
fileName = 'SCOPE_019.wdf.csv'

data = dataDict[ fileName ]

# Analysis
lorentzFwhm = 23e9      # theortical estimate from specified mirror reflectivities and clipping losses
lorentzMaxV = 0.1505312 # maximum in measurements
cavLength = 19.7e-6
cavWavelength = 637e-9
cavFreq = scipy.constants.c / cavWavelength

# Histogram
plt.hist(data[:,0],20,label=fileName)
plt.legend()
plt.grid()
plt.show()

# Vibrations
df = lorentz_voltage_to_detuning( data[:,0], time, lorentzFwhm, lorentzMaxV, doPlot=True )
dfRMS = rms_deviation( scipy.signal.detrend( df ) )
dlRMS = dfRMS * cavLength / cavFreq
print 'rms vibrations {:.3f} V, {:.3f} GHz, {:.3f} nm using Lorenzian look-up'.format( rms_deviation( data[:,0] ), dfRMS*1e-9 , dlRMS*1e9 )                





In [ ]:
# compute spectrum
dl = df * cavLength / cavFreq
freq_pow, power = scipy.signal.welch( dl, axis = 0, fs = 1/timeResolution, nperseg = len(dl), detrend = 'linear' )


In [ ]:
# integrate spectrum

min_freq = 0.1  # we do not take frequencies below 0.1 Hz into account.
max_freq = 20e3

min_ind = np.argmin(np.abs(freq_pow-min_freq))
max_ind = np.argmin(np.abs(freq_pow-max_freq))

full_pow = scipy.integrate.trapz(power,freq_pow,axis=0)
dc_pow = scipy.integrate.trapz(power[0:min_ind],freq_pow[0:min_ind],axis=0)

cum_freqs = freq_pow[min_ind+1:max_ind]      
cum_pow = scipy.integrate.cumtrapz( power[min_ind:max_ind], freq_pow[min_ind:max_ind] )
cum_pow = cum_pow / (full_pow-dc_pow)  

In [ ]:
plt.figure(figsize=(12, 12))
plt.subplot(211)
plt.plot(freq_pow[min_ind:max_ind], 10.0*np.log10( power[min_ind:max_ind] ) )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (m**2 in dB)')
plt.subplot(212)
plt.plot(cum_freqs, cum_pow)
plt.title( 'full noise power {:.3f} nm/sqrt(Hz), dc frequency {} Hz, relative DC power {:.3f}'.format( np.sqrt( full_pow ) *1e9, min_freq, dc_pow / full_pow )  )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Integrated noise power, relative (m**2)')
plt.show()

  


In [ ]:
plt.figure(figsize=(12, 12))
plt.plot( cum_freqs, plot1, label='#017, 18 GHz lw' )
plt.plot( cum_freqs, plot2, label='#017, 23 GHz lw', linestyle=':' )
plt.plot( cum_freqs, plot3, label='#019, 18 GHz lw' )
plt.plot( cum_freqs, plot4, label='#019, 23 GHz lw', linestyle=':' )
plt.xlim( 0, 1000 )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Integrated noise power, relative (m**2)')
plt.legend()
plt.show()

## Measurements 14 September 2017 - Cavity length 20 um

In [ ]:
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170918/scope'

fileNames = [ 'SCOPE_006.wdf.csv', 'SCOPE_007.wdf.csv', 'SCOPE_008.wdf.csv', 'SCOPE_009.wdf.csv', 'SCOPE_010.wdf.csv', 'SCOPE_011.wdf.csv', 'SCOPE_012.wdf.csv', 'SCOPE_020.wdf.csv' ]

# fileNames = [ 'SCOPE_006.wdf.csv', 'SCOPE_013.wdf.csv', 'SCOPE_014.wdf.csv', 'SCOPE_015.wdf.csv', 'SCOPE_016.wdf.csv', 'SCOPE_017.wdf.csv', 'SCOPE_018.wdf.csv', 'SCOPE_019.wdf.csv' ]

# dataDict = {}

for fileName in fileNames:
    if fileName in dataDict:
        continue
    data, timeResolution = read_scope_xviewer_csv( os.path.join(dataDir,fileName) )
    time = timeResolution * np.arange(data.shape[0]) 
    print 'time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
        timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )   
    dataDict[fileName] = data

In [ ]:
for fileName in fileNames:    
    data = dataDict[fileName]
    i1 = 0
    i2 = np.argmax( time > 1 )
    plt.figure(figsize=(16, 8))
    plt.plot( time[i1:i2], data[i1:i2,0] )
    plt.grid()
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'V' )
    plt.title( fileName )
    plt.show()  

for fileName in fileNames:    
    data = dataDict[fileName]    
    plt.hist(data[:,0],20,label=fileName)
plt.legend()
plt.grid()
plt.show()

for fileName in fileNames:    
    data = dataDict[fileName]    
    print 'filename {}, min {:10.3f} V, max {:10.3f} V'.format( fileName, np.amin( data[:,0] ), np.amax( data[:,0] ) )  



In [ ]:
# Compute dl and RMS values

# dlDict = {}

for fileName in fileNames[1:]:    
    data = dataDict[fileName]    
    
    # Analysis    
    lorentzFwhm = 18.0e9        # theortical estimate from specified mirror reflectivities and clipping losses
    lorentzMaxV = 0.0863000001  # maximum in measurements 
    cavLength = 19.7e-6
    cavWavelength = 637e-9
    cavFreq = scipy.constants.c / cavWavelength

    # Vibrations
    df = lorentz_voltage_to_detuning( data[:,0], time, lorentzFwhm, lorentzMaxV, doPlot=True )
    dfRMS = rms_deviation( scipy.signal.detrend( df ) )
    dlRMS = dfRMS * cavLength / cavFreq        
    print '{} (lw {:.1f} GHz) rms vibrations {:.3f} V, {:.3f} GHz, {:.3f} nm'.format( fileName, lorentzFwhm*1e-9, rms_deviation( data[:,0] ), dfRMS*1e-9 , dlRMS*1e9 )                    
                
    # dlDict[fileName] = df * cavLength / cavFreq
                
    
    
#013 0.144 nm
#014 0.149 nm
#015 0.169 nm
#016 0.166 nm

#017	0.122 nm
#018	0.127
#019	0.118

In [ ]:
# Compute spectra and cumtrapz

def calc_cum_pow( freq_power, power, min_freq=0.1, max_freq=20e3 ):

    # min_freq = 0.1  # we do not take frequencies below 0.1 Hz into account.
    # max_freq = 20e3

    min_ind = np.argmin(np.abs(freq_pow-min_freq))
    max_ind = np.argmin(np.abs(freq_pow-max_freq))

    full_pow = scipy.integrate.trapz(power,freq_pow,axis=0)
    dc_pow = scipy.integrate.trapz(power[0:min_ind],freq_pow[0:min_ind],axis=0)

    cum_freqs = freq_pow[min_ind+1:max_ind]      
    cum_pow = scipy.integrate.cumtrapz( power[min_ind:max_ind], freq_pow[min_ind:max_ind] )
    cum_pow = cum_pow / (full_pow-dc_pow)  
    
    return cum_freqs, cum_pow, full_pow, dc_pow


plt.figure(figsize=(12, 12))

if not( 'powerDict' in locals() ):
    powerDict = {}
    freqPowDict = {}  

for fileName in fileNames[1:]:    
    dl = dlDict[fileName] 
    if fileName in powerDict:
        power = powerDict[fileName]
        freqPow = freqPowDict[fileName]
    else:    
        freq_pow, power = scipy.signal.welch( dl, axis = 0, fs = 1/timeResolution, nperseg = len(dl), detrend = 'linear' )
        powerDict[fileName] = power
        freqPowDict[fileName] = freqPow
        
    cumFreqs, cumPow, fullPow, dcPow = calc_cum_pow( freqPow, power, max_freq=250  )
       
    plt.plot( cumFreqs*1e-3, cumPow, label=fileName )        
    print '{}, full noise power {:.3f} nm/sqrt(Hz), dc frequency {} Hz, relative DC power {:.3f}'.format( fileName, np.sqrt( fullPow ) *1e9, 0.1, dcPow / fullPow ) 
    
plt.grid()
plt.xlabel('freq (kHz)')
plt.ylabel('Integrated noise power, relative (m**2)')
plt.legend()
plt.show()
    